In [12]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from pandas import read_csv

import sklearn

import matplotlib.pyplot as pl
%matplotlib inline
import seaborn as sns
from scipy import sparse

import itertools
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import roc_auc_score as roc 
from sklearn import linear_model 

Считаем выборку

In [2]:
train = pd.read_csv("linear_train.txt", names=['word', 'lable'])

In [3]:
ans = pd.read_csv("linear_ans_example.txt")
ans.head()

,Id,Answer
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [4]:
test = pd.read_csv("linear_test.txt", names=['word'])
test.head()

,word
0,﻿Аалто
1,ААР
2,Аара
3,Ааре
4,Аарон


In [5]:
len(train)

101408

Добавим некоторые признаки, которые могут нам помочь: длина, количество гласных и согласных

In [6]:
train['length'] = train['word'].apply(lambda word: len(word))
train.head()

,word,lable,length
0,Аалтонен,1,8
1,Аар,0,3
2,Аарон,0,5
3,ААРОН,0,5
4,Аарона,0,6


In [7]:
vowels = [ 'а',  'я', 'ё', 'у','е', 'о', 'э', 'ю', 'и', 'ы', 'Ё', 'У', 'Е', 'Ы','А', 'О', 'Э', 'Ю', 'И', 'Я' ]
train['vowels'] = train['word'].apply(lambda word: len([letter for letter in word if letter in vowels]))
train['consonants'] = train['length'] - train['vowels']
train.head()

,word,lable,length,vowels,consonants
0,Аалтонен,1,8,4,4
1,Аар,0,3,2,1
2,Аарон,0,5,3,2
3,ААРОН,0,5,3,2
4,Аарона,0,6,4,2


In [9]:
test['vowels'] = test['word'].apply(lambda word: len([letter for letter in word if letter in vowels]))
test['length'] = test['word'].apply(lambda word: len(word))
test['consonants'] = test['length'] - test['vowels']
test.head()

,word,vowels,length,consonants
0,﻿Аалто,3,6,3
1,ААР,2,3,1
2,Аара,3,4,1
3,Ааре,3,4,1
4,Аарон,3,5,2


Посчитаем частоту вхождения n-грам для n от 3 до 5

In [10]:
CV = CountVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=5)

CV.fit(list(train.word) + list(test.word))
ngram_train = CV.transform(train.word)
ngram_test = CV.transform(test.word)

In [16]:
X_train = sparse.hstack([ngram_train, train["vowels"].reshape(-1, 1)])

In [17]:
X_train = sparse.hstack([X_train, train["length"].reshape(-1, 1)])

In [18]:
X_train = sparse.hstack([X_train, train["consonants"].reshape(-1, 1)])

In [19]:
X_test = sparse.hstack([ngram_test, test["vowels"].reshape(-1, 1)])

In [20]:
X_test = sparse.hstack([X_test, test["length"].reshape(-1, 1)])

In [21]:
X_test = sparse.hstack([X_test, test["consonants"].reshape(-1, 1)])

Применим к полученным признакам логистическую регрессию

In [22]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, train.lable)
pred = lr.predict_proba(X_test)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(ngram_train, train.lable, train_size=.8) 

In [16]:
X_test[:10]

<10x123757 sparse matrix of type '<class 'numpy.int64'>'
	with 206 stored elements in Compressed Sparse Row format>

In [23]:
prediction = lr.predict_proba(X_test)[:,1]
print(roc(Y_test, prediction))

NameError: name 'Y_test' is not defined

Оценим количество слов из тестовой выборки, совпадающее с количеством слов из трэйна

In [24]:
len(set(train['word']).intersection(set(test['word'])))

2785

При этом длина теста 

In [25]:
len(test)

188920

Заменим значения, полученные логрегрессией на значения из трейна для всех этих слов

In [26]:
ind_dict = dict((k,i) for i,k in enumerate(train['word']))
inter = set(ind_dict).intersection(set(test['word']))
indices = [ ind_dict[x] for x in inter ]

In [40]:
test['length'] = test['word'].apply(lambda word: len(word))
test['vowels'] = test['word'].apply(lambda word: len([letter for letter in word if letter in vowels]))
test['consonants'] = test['length'] - test['vowels']

In [85]:
len(indices)

2785

In [27]:
%%time
submit1 = pd.DataFrame()
submit1['Id'] = range(len(test))
submit1['Answer'] = pred[:,1]
for i in indices:
    submit1['Answer'][np.array(test.loc[test['word'] == train['word'][i]].index)[0]] = float(1 - train['lable'][i])

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CPU times: user 2min 33s, sys: 290 ms, total: 2min 33s
Wall time: 2min 34s


Запишем результат в файл

In [31]:
submit1.to_csv('submit_3.csv', index=None)

In [19]:
submit1.head()

,Id,Answer
0,0,0.364908
1,1,0.138456
2,2,0.158695
3,3,0.043595
4,4,1.000000
